In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import geopandas as gpd
import mercantile
from shapely.geometry import box

import os
os.chdir("/home/gridsan/qwang/urban-control/")

from src.utils_io import *
from src.utils_geo import *

In [30]:
city = 'chicago'

params_file = f"configs/configs_{city}.yaml"
params = load_config(params_file)
io_file = f"configs/io_{city}.yaml"
io_config = load_config(io_file)
osm_params_file = "configs/configs_osm_layers.yaml"
osm_params = load_config(osm_params_file)

In [6]:
# Create tile grid
bbox = [float(x) for x in params['bounding_box'].split(',')]
tile_grid = create_tile_grid(bbox, params['zoom'])

# Calculate tile area
tile_area = calculate_tile_area(params['zoom'], params['central_x'], params['central_y'])
tile_area = np.round(tile_area, -2)

In [10]:
layer_render = osm_params['layers_render'].keys()

# Read relevant shapefiles
load_files = {}
for name in layer_render:
    load_files[name] = io_config['geojsons'][name]
load_files['landuse_complement'] = io_config['geojsons']['landuse_complement']
layers = load_all(load_files, params={'bbox':bbox})


Loading ...  water
Loading ...  waterways
Loading ...  railways
Loading ...  roads
Loading ...  landuse_complement


In [13]:
from multiprocessing import Pool
NUM_WORKERS = 8

grouped_layers = {}
print("📦 Loading and assigning layers...")

layer = layers['landuse_complement'].to_crs("EPSG:3857")
layer = layer[layer['fclass']=='industrial']
assigned = assign_features_to_tiles(layer, tile_grid)
grouped_layers['industrial'] = assigned

for name in layer_render:
    layer = layers[name].to_crs("EPSG:3857")
    if 'filters' in osm_params['layers_render'][name]:
        for c, f in osm_params['layers_render'][name]['filters'].items():
            if isinstance(f, list):
                layer = layer[layer[c].isin(f)]
            elif isinstance(f, dict):
                if 'geq' in f.keys():
                    layer = layer[layer[c] >= float(f['geq'])]
            else:
                raise ValueError("unknown filter condition")
    assigned = assign_features_to_tiles(layer, tile_grid)
    grouped_layers[name] = assigned
    print(f"  ✅ {name}: {len(assigned)} features assigned to tiles")

📦 Loading and assigning layers...
  ✅ water: 42489 features assigned to tiles
  ✅ waterways: 25039 features assigned to tiles
  ✅ railways: 37381 features assigned to tiles
  ✅ roads: 109655 features assigned to tiles


In [43]:
tile_grid.merge(grouped_layers['industrial'][['z','x','y']].drop_duplicates(), on=['z','x','y'])

,z,x,y,xmin,ymin,xmax,ymax,geometry
0,16,16620,24303,-9.874441e+06,5.175704e+06,-9.873830e+06,5.176316e+06,"POLYGON ((-9873829.566 5175704.059, -9873829.5..."
1,16,16620,24329,-9.874441e+06,5.159805e+06,-9.873830e+06,5.160417e+06,"POLYGON ((-9873829.566 5159805.157, -9873829.5..."
2,16,16620,24334,-9.874441e+06,5.156748e+06,-9.873830e+06,5.157359e+06,"POLYGON ((-9873829.566 5156747.676, -9873829.5..."
3,16,16620,24335,-9.874441e+06,5.156136e+06,-9.873830e+06,5.156748e+06,"POLYGON ((-9873829.566 5156136.180, -9873829.5..."
4,16,16620,24336,-9.874441e+06,5.155525e+06,-9.873830e+06,5.156136e+06,"POLYGON ((-9873829.566 5155524.684, -9873829.5..."
...,...,...,...,...,...,...,...,...
6428,16,16835,24438,-9.742969e+06,5.093152e+06,-9.742358e+06,5.093764e+06,"POLYGON ((-9742357.877 5093152.069, -9742357.8..."
6429,16,16835,24439,-9.742969e+06,5.092541e+06,-9.742358e+06,5.093152e+06,"POLYGON ((-9742357.877 5092540.572, -9742357.8..."
6430,16,16835,24440,-9.742969e+06,5.091929e+06,-9.742358e+06,5.092541e+06,"POLYGON ((-9742357.877 5091929.076, -9742357.8..."
6431,16,16835,24449,-9.742969e+06,5.086426e+06,-9.742358e+06,5.087037e+06,"POLYGON ((-9742357.877 5086425.610, -9742357.8..."


In [36]:
sample_grid = tile_grid[(tile_grid['x'] == 16814) & (tile_grid['y'] == 24363)]

render_tile((16, 16814, 24363, sample_grid['geometry'].iloc[0], 
             grouped_layers, osm_params['layers_render'], "tmp/"))

water
waterways
railways
roads
industrial


In [ ]:
print("🎨 Rendering tiles in parallel...")
tile_jobs = [
    (row["z"], row["x"], row["y"], row["geometry"], grouped_layers, params['layers_render'], tile_save_dir)
    for _, row in tile_grid.iterrows()
]

with Pool(NUM_WORKERS) as pool:
    pool.map(render_tile, tile_jobs)

print("✅ Done! Tiles saved to:", OUTPUT_DIR)